In [1]:
puzzle = open("AoC21.txt").read().splitlines()
puzzle = [list(x) for x in puzzle]

In [2]:
def convertDiffToMoves(coord1, coord2, pos):
    a,b = coord1
    c,d = coord2
    corns = set()
    if (a,d) in pos:
        corns.add((a,d))
    if (c,b) in pos:
        corns.add((c,b))
    if a==c or b==d:
        return [convertDiff((c-a,d-b))]
    diffMoves = []
    for co in corns:
        diff = ""
        diff1 = (co[0]-a, co[1]-b)
        diff+=convertDiff(diff1)
        diff2 = (c-co[0], d-co[1])
        diff+=convertDiff(diff2)
        diffMoves.append(diff)
    return diffMoves

def convertDiff(diff):
    if diff[0]==0:
        if diff[1]>0:
            return ">"*diff[1]
        else:
            return "<"*(-diff[1])
    else:
        if diff[0]>0:
            return "v"*diff[0]
        else:
            return "^"*(-diff[0])

In [3]:
numPosMap = {"7": (0,0), "8": (0,1), "9": (0,2), "4": (1,0), "5": (1,1), "6": (1,2), "1": (2,0), "2": (2,1), "3": (2,2), "0": (3,1), "A": (3,2)}
numPos = set()
for k, v in numPosMap.items():
    numPos.add(v)
numPadMap = {}
for i in numPosMap:
    for j in numPosMap:
        if i!=j:
            diffs = convertDiffToMoves(numPosMap[i], numPosMap[j], numPos)
            numPadMap[(i,j)] = diffs
        else:
            numPadMap[(i,j)] = [""]

In [4]:
dirPosMap = {"^": (0,1), "A": (0,2), "<": (1,0), "v": (1,1), ">": (1,2)}
dirPos = set()
for k, v in dirPosMap.items():
    dirPos.add(v)
dirPadMap = {}
for i in dirPosMap:
    for j in dirPosMap:
        if i!=j:
            diffs = convertDiffToMoves(dirPosMap[i], dirPosMap[j], dirPos)
            dirPadMap[(i,j)] = diffs
        else:
            dirPadMap[(i,j)] = [""]

In [5]:
def genPathsFromNums(combo):
    start = "A"
    paths = [""]
    for num in combo:
        paths = [path+diff+"A" for path in paths for diff in numPadMap[(start,num)]]
        start = num
    return paths

In [6]:
def buildSeq(keys, index, prevKey, currPath, result):
    if index == len(keys):
        result.append(currPath)
        return result
    paths = dirPadMap[(prevKey, keys[index])]
    final = []
    for path in paths:
        final += buildSeq(keys, index+1, keys[index], currPath + path + 'A', result)
    return final

In [7]:
from tqdm import tqdm
def scuffedRobotSolve(paths, depth):
    if depth==0:
        return paths 
    newpaths = []
    for path in paths:
        newpaths += buildSeq(list(path), 0, "A", "", [])
    minLength = min([len(x) for x in newpaths])
    returnPaths = [x for x in newpaths if len(x)==minLength]
    return scuffedRobotSolve(returnPaths, depth-1)
def scuffedPart1Solve(combo):
    paths = genPathsFromNums(combo)
    solve = scuffedRobotSolve(paths, 2)
    #print(solve[0])
    return len(solve[0])
ans1 = 0
for i in puzzle:
    numPart = int("".join(i)[:-1])
    leng = scuffedPart1Solve(i)
    #print(i, leng)
    ans1 += leng*numPart
print(ans1)

215374


In [8]:
def splitPath(path):
    paths = []
    start = 0
    for i in range(len(path)):
        if path[i]=="A":
            paths.append(path[start:i+1])
            start = i+1
    return paths

In [9]:
def regularRobotSolve(path, depth, cache={}):
    #finds min length of a path given depth amount of robots
    if path=="":
        return 0
    #print(path)
    if depth==0:
        return len(path)
    if (tuple(path), depth) in cache:
        return cache[(tuple(path), depth)]
    paths = splitPath(path)
    minAns = 0
    for p in paths:
        possSeq = buildSeq(list(p), 0, "A", "", [])
        minVal = float("inf")
        for seq in possSeq:
            solve = regularRobotSolve(seq, depth-1, cache)
            minVal = min(minVal, solve)
        minAns += minVal
    cache[(tuple(path), depth)] = minAns
    return minAns

In [46]:
def unscuffedPart2Solve(combo):
    paths = genPathsFromNums(combo)
    minSolve = float("inf")
    for path in paths:
        minSolve = min(minSolve, regularRobotSolve(path, 25))
    #print(solve[0])
    return minSolve
ans2 = 0
for i in puzzle:
    numPart = int("".join(i)[:-1])
    leng = unscuffedPart2Solve(i)
    #print(i, leng)
    ans2 += leng*numPart
print(ans2)

260586897262600
